# Recherche et alignement, outils de cartographie et de correspondance des bases de données :

- Base Joconde
- Gallica
- Europeana





BaOIA


In [ ]:
#@title Installation des bibliothèques nécessaires et connexion à un compte Google Drive.

''' 
Google Colab notebook.
Python == 3.7.11

BaOIA - La Contemporaine - Université de Nanterre
'''


## Installation des bibliothèques et connexion au compte Google Drive

!pip install utils
from google.colab import drive
import json
from openpyxl import load_workbook
import pandas as pd
import re
import requests
!pip install langdetect
from langdetect.lang_detect_exception import LangDetectException
import shutil
!pip install xmltojson
import xmltodict
import urllib
import urllib.request, urllib.error, urllib.parse
import xmltojson
from langdetect import detect
import os
import xml.etree.ElementTree as ET
import lxml
from collections import OrderedDict
from lxml import etree
!pip install unidecode==1.2.0
import unidecode
import folium
import unicodedata
from urllib.error import HTTPError
from re import match
from bs4 import BeautifulSoup
import urllib.request, urllib.error, urllib.parse
import string

drive.mount('/content/drive/')


#@markdown Lors du lancement de la cellule, la connexion vers le Google Drive sera nécessaire, les étapes à suivre sont:

#@markdown - Indiquer le chemin absolu vers le dossier de travail contenu sur le Google Drive (ici, le dossier de travail se nomme "dossier_de_travail_MonumenTAL")
chemin_absolu_vers_le_dossier_de_travail = '/content/drive/My Drive/BaOIA/api_bases_de_donnees/' #@param {type:"string"}



#@markdown - En dessous de la cellule, un lien bleu va apparaître après 'Go to this URL in a browser'. Cliquer dessus, un nouvel onglet du navigateur va s'ouvrir.

#@markdown - Sur ce nouvel onglet, cliquer sur le compte Google auquel le Drive est associé. Cliquer ensuite sur "Connexion" puis sur la petite icône en forme de carré pour copier le lien.

#@markdown - De retour sur le notebook de travail, coller le lien dans la case en dessous de la cellule après "Enter your authorization code:" puis appuyer sur la touche "Enter" du clavier. La cellule va finir de l'exécuter normalement.
if not os.path.exists(chemin_absolu_vers_le_dossier_de_travail):
    os.makedirs(chemin_absolu_vers_le_dossier_de_travail)
os.chdir(chemin_absolu_vers_le_dossier_de_travail)


In [4]:
#@title Recherche de données correspondantes à partir d'un terme dans les trois bases de données.

#@markdown

#@markdown Indiquer ici le terme à chercher dans les trois bases de données (oeuvres, artistes)

terme_a_chercher = "Venus de Milo" #@param {type:"string"}
os.chdir(chemin_absolu_vers_le_dossier_de_travail)
os.makedirs(terme_a_chercher, exist_ok=True)
os.chdir(terme_a_chercher)


## Recherche dans la base Joconde:
if ' ' in terme_a_chercher:
  nom_pour_lien = re.sub(r' ', '%20', terme_a_chercher)
  lien_de_recherche_joconde = "".join("https://data.culture.gouv.fr/api/records/1.0/search/?dataset=base-joconde-extrait&q="+nom_pour_lien+'&rows=10000')
else: 
  lien_de_recherche_joconde="".join("https://data.culture.gouv.fr/api/records/1.0/search/?dataset=base-joconde-extrait&q="+terme_a_chercher+'&rows=10000')
request = urllib.request.Request(lien_de_recherche_joconde)
try:
  content = urllib.request.urlopen(request)
except HTTPError as he:
  print('Pas de téléchargement possible pour la base Joconde avec cette recherche')
parse = BeautifulSoup(content,'html.parser')
json_infos=json.loads(parse.text)
#@markdown Indiquer ici le nom du fichier qui sera créée avec les informations tirées de la base Joconde (le fichier doit terminer par .json : c'est le format du fichier de données)
nom_du_fichier_json_base_joconde = 'donnees_base_joconde.json' #@param {type:"string"}
with open(nom_du_fichier_json_base_joconde, 'w') as ny:
  json.dump(json_infos, ny, indent=4, ensure_ascii=False)



## Recherche base Gallica:
if ' ' in terme_a_chercher:
  nom_pour_lien = re.sub(r' ', '%20', terme_a_chercher)
  lien_de_recherche_gallica = "".join("https://gallica.bnf.fr/SRU?version=1.2&operation=searchRetrieve&query=dc.title%20any%20%22"+nom_pour_lien+'%22&maximumRecords=49&deway=%20any%207')
else:
  lien_de_recherche_gallica = "".join("https://gallica.bnf.fr/SRU?version=1.2&operation=searchRetrieve&query=dc.title%20any%20%22"+terme_a_chercher+'%22&maximumRecords=49&deway=%20any%207')
request = urllib.request.Request(lien_de_recherche_gallica)
try:
  content = urllib.request.urlopen(request)
except HTTPError as he:
  print('Pas de téléchargement possible pour la base Gallica avec cette recherche')
parse = BeautifulSoup(content,'html.parser')
with open ('fichier_temporaire.xml', 'w') as pu:
  pu.write(parse.prettify())
with open('fichier_temporaire.xml', 'r') as po:
  my =xmltodict.parse(po.read())
  my2 = dict(OrderedDict(my))
#@markdown Indiquer ici le nom du fichier qui sera créée avec les informations tirées de la base Gallica (le fichier doit terminer par .json : c'est le format du fichier de données)
nom_du_fichier_json_base_gallica = 'donnees_base_gallica.json' #@param {type:"string"}
with open(nom_du_fichier_json_base_gallica, 'w') as pi:
  json.dump(my2, pi, indent=4, ensure_ascii=False)
  a_suppr = "".join(chemin_absolu_vers_le_dossier_de_travail+'/'+terme_a_chercher+'/fichier_temporaire.xml')
os.remove(a_suppr)


## Recherche base Europeana:
if ' ' in terme_a_chercher:
  nom_pour_lien = re.sub(r' ', '%20', terme_a_chercher)
  lien_de_recherche_europeana = "".join("https://api.europeana.eu/record/v2/search.json?query="+nom_pour_lien+'&rows=100&theme=art&wskey=alemalle')
else:
  lien_de_recherche_europeana = "".join("https://api.europeana.eu/record/v2/search.json?query="+terme_a_chercher+'&rows=100&theme=art&wskey=alemalle')
request = urllib.request.Request(lien_de_recherche_europeana)
try:
  content = urllib.request.urlopen(request)
except HTTPError as he:
  print('Pas de téléchargement possible pour la base Europeana avec cette recherche')
parse = BeautifulSoup(content,'html.parser')
json_infos=json.loads(parse.text)
#@markdown Indiquer ici le nom du fichier qui sera créée avec les informations tirées de la base Europeana (le fichier doit terminer par .json : c'est le format du fichier de données)
nom_du_fichier_json_base_europeana = 'donnees_base_europeana.json' #@param {type:"string"}
with open(nom_du_fichier_json_base_europeana, 'w') as mu:
  json.dump(json_infos, mu, indent=4, ensure_ascii=False)


#@markdown Il est possible de jeter un oeil aux trois fichiers créés qui contiennent déjà les informations "brutes" tirées des bases de données.



In [5]:
#@title Création de fichiers de données avec les informations sélectionnées et téléchargement des images correspondantes.
chemin_dossier = "".join(chemin_absolu_vers_le_dossier_de_travail+'/'+terme_a_chercher)
os.chdir(chemin_dossier)
#@markdown Informations conservées dans les nouveaux fichiers : titre de l'oeuvre, auteur/artiste, date de création, domaine/sujet, matériaux.
## Téléchargement base Europeana
#@markdown Indiquer ici le nom du dossier dans lesquelles les images de la base Europeana vont se télécharger:.
dossier_base_europeana = 'images_base_europeana' #@param {type:"string"}
new_europeana = {}
auteurs, typee, sujets=([] for i in range(3))
chemin_fichier = "".join(chemin_absolu_vers_le_dossier_de_travail+'/'+terme_a_chercher+'/'+nom_du_fichier_json_base_europeana)
import ssl

ssl._create_default_https_context = ssl._create_unverified_context
with open(chemin_fichier, 'r') as pl:
  europeana = json.load(pl)
for keys, values in europeana.items():
  if keys == 'items':
    for ele in values:
      for keys2, values2 in ele.items():
        if keys2 == 'title':
          titre_trouve = str(values2)
          nouveau_titre_trouve = re.sub(r"[\['\]]",'',  titre_trouve)
        if keys2 == 'dcCreator':
          auteurs = str(values2)
        if keys2 == 'edmConceptPrefLabelLangAware':
          for keys3, values3 in values2.items():
            if keys3 == 'fr':
              typee = str(values3)
        if keys2 == 'edmConceptLabel':
          for ele2 in values2:
            for keys4, values4 in ele2.items():
              try:
                lang = detect(values4)
              except LangDetectException as lde:
                pass
              if lang == 'fr':
                sujets = str(values4)
        if keys2 == 'edmIsShownBy':
          for ele2 in values2:
            lien = ele2
      new_europeana[nouveau_titre_trouve]= {'Auteur': auteurs, 'Type': typee, 'Sujet': sujets, 'Lien': lien}
#@markdown Indiquer ici le nom du fichier de données nettoyé contenant uniquement les informations essentielles, base Europeana. Le fichier est au format json.
nom_du_fichier_nettoyé_base_europeana = 'base_Europeana_final.json' #@param {type:"string"}
print('Création du nouveau fichier nettoyé pour la base Europeana.')
with open(nom_du_fichier_nettoyé_base_europeana, 'w') as tu:
  json.dump(new_europeana, tu, indent=4, ensure_ascii=False)

if not os.path.exists(dossier_base_europeana):
  os.mkdir(dossier_base_europeana)
os.chdir(dossier_base_europeana)
for rerekey, rerevalue in new_europeana.items():
  rerekey = re.sub(r"[/;:""']", "_", rerekey)
  if len(rerekey) >= 100:
    rerekey = rerekey[:100]
  nom_image2 = "".join(rerekey + '.jpg')
  for rerekey2, rerevalue2 in rerevalue.items():
    if rerekey2 == 'Lien':
      if ' ' in rerevalue2:
        rerevalue2 = re.sub(r' ', '%20', rerevalue2)
      try:
        urllib.request.urlretrieve(rerevalue2, nom_image2)
      except (OSError, HTTPError, urllib.error.ContentTooShortError) as http:
        pass





## BASE JOCONDE
os.chdir(chemin_dossier)
new_joconde = {}
auteurss, sujetss,materiaux_techniques, datess = ([] for i in range(4))
with open(nom_du_fichier_json_base_joconde, 'r') as pi:
  donneess = json.load(pi)
for key, value in donneess.items():
  if key == 'records':
    for ele in value:
      for key2, value2 in ele.items():
        if key2 == 'fields':
          for key3, value3 in value2.items():
            if key3 == 'titre':
              titre = str(value3)
            if key3 == 'auteur':  
              auteurss = value3          
            if key3 == 'materiaux_techniques':  
              materiaux_techniques = value3          
            if key3 == 'sujet_represente':
              sujetss = value3
            if key3 == 'epoque':
              datess = str(value3)
            if key3 == 'periode_de_creation':
              datess = value3
      new_joconde[titre] = {'Date': datess, 'Auteur':auteurss, 'Matériaux': materiaux_techniques, 'Sujet': sujetss}
print('Création du nouveau fichier nettoyé pour la base Joconde.')
#@markdown Indiquer ici le nom du fichier de données nettoyé contenant uniquement les informations essentielles, base Joconde. Le fichier est au format json.
nom_du_fichier_nettoyé_base_joconde = 'base_Joconde_final.json' #@param {type:"string"}
with open(nom_du_fichier_nettoyé_base_joconde, 'w') as pl:
  json.dump(new_joconde, pl, indent=4, ensure_ascii=False)


### Base GALLICA
#@markdown Indiquer ici le nom du dossier dans lesquelles les images de la base Gallica vont se télécharger:
dossier_base_gallica = 'images_base_gallica' #@param {type:"string"}
new_gallica = {}
chemin_fichier2 = "".join(chemin_dossier+'/'+nom_du_fichier_json_base_gallica)
with open(chemin_fichier2, 'r') as ret:
  gallica = json.load(ret)
print("Les images sont en train de se télécharger...")
for ele in gallica['srw:searchretrieveresponse']['srw:records']['srw:record']:
  for key, value in ele.items():
    if key == 'srw:recorddata':
      for key2, value2 in value.items():
        for key3, value3 in value2.items():
          if key3 == 'dc:title':
            le_titre = str(value3)
            le_nouveau_titre = re.sub(r"[\['\]]", '', le_titre)
            new_gallica[le_nouveau_titre] = {}
          if key3 == 'dc:date':
            date = value3
          if key3 == 'dc:creator':
            auteur = value3
          if key3 == 'dc:type':
            types = value3
          if key3 == 'dc:subject':
            subject = value3
          if key3 == 'dc:identifier':
            if type(value3) == list:
              value3 = value3[1]
            identifiant = value3[27:]
            lien = 'http://gallica.bnf.fr/iiif/ark:' + identifiant + '/f1/full/3000/0/native.jpg'
  new_gallica[le_nouveau_titre] = {'Date': date, 'Auteur':auteur, 'Type': types, 'Sujet': subject, 'Lien': lien}
os.chdir(chemin_dossier)
#@markdown Indiquer ici le nom du fichier de données nettoyé contenant uniquement les informations essentielles, base Gallica. Le fichier est au format json.
print('Création du nouveau fichier nettoyé pour la base Gallica.')
nom_du_fichier_nettoyé_base_gallica = 'base_Gallica_final.json' #@param {type:"string"}
with open(nom_du_fichier_nettoyé_base_gallica, 'w') as pl:
  json.dump(new_gallica, pl, indent=4, ensure_ascii=False)

if not os.path.exists(dossier_base_gallica):
  os.mkdir(dossier_base_gallica)
os.chdir(dossier_base_gallica)
for rekey, revalue in new_gallica.items():
  rekey = re.sub(r"[/;:']", "_", rekey)
  nom_image = "".join(rekey + '.jpg')
  for rekey2, revalue2 in revalue.items():
    if rekey2 == 'Lien':
      if ' ' in revalue2:
        revalue2 = re.sub(r' ', '%20', revalue2)
      try:
        urllib.request.urlretrieve(revalue2, nom_image)
      except (OSError, HTTPError, urllib.error.ContentTooShortError) as http:
        pass


Création du nouveau fichier nettoyé pour la base Europeana.
Création du nouveau fichier nettoyé pour la base Joconde.
Les images sont en train de se télécharger...
Création du nouveau fichier nettoyé pour la base Gallica.


In [6]:
#@title Outil de repérages des oeuvres (titres) identiques entre les bases de données (pour identifier les reprises d'autres artistes, des dates de créations ensuite différentes...)


#@markdown Les noms sont trouvés uniquement si les valeurs sont identiques, au charactère près; il y a donc peut-être d'autres oeuvres identiques qui ne sont pas affichées ci-dessous.

setA = set(new_joconde)
setB = set(new_europeana)
setC = set(new_gallica)
 
print('Les oeuvres identiques entre la Base Joconde et Gallica: ',setA.intersection(setC))
print('Les oeuvres identiques entre Europeana et Gallica: ',setB.intersection(setC))
print('Les oeuvres identiques entre la Base Joconde et Europeana: ',setA.intersection(setB))

Les oeuvres identiques entre la Base Joconde et Gallica:  set()
Les oeuvres identiques entre Europeana et Gallica:  {'La Vénus de Milo / par Félix Ravaisson,...', 'Vénus de Milo : estampe / Lancrenon del. ; Z. Prévost', 'La Vénus de Milo / par P. Morey,...'}
Les oeuvres identiques entre la Base Joconde et Europeana:  set()


In [ ]:
#@title Représentation cartographique des lieux de conservation des artefacts repérés dans la base de données JOCONDE (les musées dans lequels les objets sont conservés, les villes)


#@markdown 
os.chdir(chemin_dossier)
donnees = {}
localisation, latitude, longitude = ([] for i in range(3))
with open(nom_du_fichier_json_base_joconde, 'r') as pi:
  donneess = json.load(pi)
for key, value in donneess.items():
  if key == 'records':
    for ele in value:
      for key2, value2 in ele.items():
        if key2 == 'fields':
          for key3, value3 in value2.items():
            if key3 == 'titre':
              titre = value3
              titre = re.sub(r"[\['\];\-/]", "", titre)
              donnees[titre] = {}
            try:
              if key3 == 'localisation':
                localisation= value3
                donnees[titre]['localisation']= localisation
              if key3 == 'geolocalisation_ville':
                new_ele = str(value3[0])
                donnees[titre]['longitude'] =new_ele
                new_ele2 = str(value3[1])
                donnees[titre]['latitude']=new_ele2
            except KeyError as ke:
              pass

## Création de la carte:
map = folium.Map(location=[48.856614, 2.3522219], zoom_start=5)
for keys, values in donnees.items():
  for keys2, values2 in values.items():
    if keys2 == 'longitude':
      lat = values2
    if keys2 == 'latitude':
      longi = values2
  #  try:
      folium.Marker([float(lat), float(longi)]).add_child(folium.Popup(keys)).add_to(map) 
    #except TypeError as terr:
   #   pass
#@markdown Indiquer ici le nom de la carte au format .html qui sera créée:
nom_de_la_carte = 'carte_lieux_conservation_base_joconde.html' #@param {type:"string"}
map.save(nom_de_la_carte)
map


#@markdown Il est possible de télécharger depuis le drive la carte créée et de l'ouvrir dans le navigateur. La carte est intéractive et indique le nom de l'oeuvre quand on clique sur chaque localisation.